# Interpretation of BertForSequenceClassification in captum

In [2]:
from captum.attr import visualization as viz
from captum.attr import IntegratedGradients, LayerConductance, LayerIntegratedGradients
from captum.attr import configure_interpretable_embedding_layer, remove_interpretable_embedding_layer

from detector import Detector
import torch

In [3]:
# import sys
#
# print(sys.executable)
# model2 = BertForSequenceClassification.from_pretrained('../model/')


In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:

# load model
model = Detector("./base_9.pt")
model.model.to(device)
model.model.eval()
model.model.zero_grad()

# load tokenizer
tokenizer = model.tokenizer

Initializing Detector...


In [6]:
def predict(inputs):
    return model.model(inputs)[0]

In [7]:
ref_token_id = tokenizer.pad_token_id # A token used for generating token reference
sep_token_id = tokenizer.sep_token_id # A token used as a separator between question and text and it is also added to the end of the text.
cls_token_id = tokenizer.cls_token_id # A token used for prepending to the concatenated question-text word sequence

In [8]:
def construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id):

    text_ids = tokenizer.encode(text, add_special_tokens=False)
    # construct input token ids
    input_ids = [cls_token_id] + text_ids + [sep_token_id]
    # construct reference token ids 
    ref_input_ids = [cls_token_id] + [ref_token_id] * len(text_ids) + [sep_token_id]

    return torch.tensor([input_ids], device=device), torch.tensor([ref_input_ids], device=device), len(text_ids)

def construct_input_ref_token_type_pair(input_ids, sep_ind=0):
    seq_len = input_ids.size(1)
    token_type_ids = torch.tensor([[0 if i <= sep_ind else 1 for i in range(seq_len)]], device=device)
    ref_token_type_ids = torch.zeros_like(token_type_ids, device=device)# * -1
    return token_type_ids, ref_token_type_ids

def construct_input_ref_pos_id_pair(input_ids):
    seq_length = input_ids.size(1)
    position_ids = torch.arange(seq_length, dtype=torch.long, device=device)
    # we could potentially also use random permutation with `torch.randperm(seq_length, device=device)`
    ref_position_ids = torch.zeros(seq_length, dtype=torch.long, device=device)

    position_ids = position_ids.unsqueeze(0).expand_as(input_ids)
    ref_position_ids = ref_position_ids.unsqueeze(0).expand_as(input_ids)
    return position_ids, ref_position_ids
    
def construct_attention_mask(input_ids):
    return torch.ones_like(input_ids)

def custom_forward(inputs):
    preds = predict(inputs)
    return torch.softmax(preds, dim = 1)[0][0].unsqueeze(-1)

def summarize_attributions(attributions):
    attributions = attributions.sum(dim=-1).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    return attributions

lig = LayerIntegratedGradients(custom_forward, model.model.roberta.embeddings)

In [13]:
# model.model(input_ids)

In [14]:
# predict(input_ids)

tensor([0.0032], device='cuda:0', grad_fn=<UnsqueezeBackward0>)

In [38]:
NUM_OF_TEXT = 10
MUTATION, REAL, SYNTHETIC = 0, 1, 2
FILE_TYPE = SYNTHETIC
# DATA_FILE = './data/Test_WikiHumanQuarterSet.json'
# DATA_FILE = './data/Test_WikiMutationFullReplaceAntonyms.json'
# DATA_FILE = './data/Test_WikiMutationFullReplaceRandomWords.json'
# DATA_FILE = './data/Test_WikiMutationFullReplaceSynonyms.json'
# DATA_FILE = './data/Test_WikiMutationFullSet.json'
# DATA_FILE = './data/Test_WikiMutationFullSetDeleteArticles.json'
# DATA_FILE = './data/Test_WikiMutationFullSetMisspellings.json'
# DATA_FILE = './data/Test_WikiMutationFullSetReplaceAE.json'
# DATA_FILE = './data/Test_WikiMutationQuarterSet.json'
# DATA_FILE = './data/Test_WikiSyntheticFullSet.json'
DATA_FILE = './data/Test_WikiSyntheticQuarterSet.json'

In [39]:
from utils2 import load_standard_json
import random
text_list = load_standard_json(DATA_FILE, True)
random.shuffle(text_list)

In [40]:
#Green most important, red least important
print('\033[1m', 'Visualization For Score', '\033[0m')
for text in text_list[:NUM_OF_TEXT]:
    input_ids, ref_input_ids, sep_id = construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id)
    token_type_ids, ref_token_type_ids = construct_input_ref_token_type_pair(input_ids, sep_id)
    position_ids, ref_position_ids = construct_input_ref_pos_id_pair(input_ids)
    attention_mask = construct_attention_mask(input_ids)

    indices = input_ids[0].detach().tolist()
    all_tokens = tokenizer.convert_ids_to_tokens(indices)

    custom_forward(input_ids)

    attributions, delta = lig.attribute(inputs=input_ids,
                                        baselines=ref_input_ids,
                                        return_convergence_delta=True,
                                        internal_batch_size=1)

    score = predict(input_ids)

    attributions_sum = summarize_attributions(attributions)

    # storing couple samples in an array for visualization purposes
    all_tokens = [token.replace("Ġ", "") for token in all_tokens]
    score_vis = viz.VisualizationDataRecord(
                            attributions_sum,
                            torch.softmax(score, dim = 1)[0][0],
                            torch.argmax(torch.softmax(score, dim = 1)[0]),
                            FILE_TYPE,
                            text,
                            attributions_sum.sum(),
                            all_tokens,
                            delta)

    viz.visualize_text([score_vis])

 Visualization For Score 
